In [3]:
from bs4 import BeautifulSoup

# อ่านไฟล์ HTML
with open('/Users/apichet/Downloads/เช็คเบี้ยประกันรถยนต์ เปรียบเทียบ รู้ราคาทันที ซื้อง่าย.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# สร้าง BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# ดึงข้อมูลประกันภัย
packages = []
cards = soup.find_all('div', class_='card')

for card in cards:
    package_name = card.find('h1', class_='card-title').get_text(strip=True)
    price = card.find('h1', class_='card-title').get_text(strip=True)
    insurance_company = card.find('img', class_='align-top')['alt'] if card.find('img', class_='align-top') else 'N/A'
    car_model = card.find('div', class_='card-subtitle').get_text(strip=True) if card.find('div', class_='card-subtitle') else 'N/A'
    coverage = 'N/A'  # เนื่องจากไม่ได้ระบุข้อมูลคุ้มครองในตัวอย่าง

    packages.append({
        'package_name': package_name,
        'insurance_company': insurance_company,
        'price': price,
        'coverage': coverage,
        'car_model': car_model
    })

# สร้าง HTML ตาราง
html_table = '''
<table border="1">
    <tr>
        <th>Package Name</th>
        <th>Insurance Company</th>
        <th>Price</th>
        <th>Coverage</th>
        <th>Car Model</th>
    </tr>
'''

for package in packages:
    html_table += f'''
    <tr>
        <td>{package['package_name']}</td>
        <td>{package['insurance_company']}</td>
        <td>{package['price']}</td>
        <td>{package['coverage']}</td>
        <td>{package['car_model']}</td>
    </tr>
    '''

html_table += '</table>'

# แสดงผล HTML ตาราง
from IPython.core.display import display, HTML
display(HTML(html_table))

/var/folders/6p/zr1pt4fd6q3fpxb61scy6_rm0000gn/T/ipykernel_95361/2712636354.py:55: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Package Name,Insurance Company,Price,Coverage,Car Model


In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import json

# Load the HTML content from the provided file
file_path = '/Users/apichet/Downloads/เช็คเบี้ยประกันรถยนต์ เปรียบเทียบ รู้ราคาทันที ซื้อง่าย.html'
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Parse the HTML content
soup = BeautifulSoup(content, 'html.parser')

# Step 3: Extract the required details
insurance_plans = []

# Adjusting based on observed HTML structure
plan_cards = soup.find_all('div', class_='card fn-card-1 mb24')

for card in plan_cards:
    plan = {}
    try:
        plan['package_name'] = card.find('h1', class_='card-title d-inline-block').text.strip()
    except AttributeError:
        plan['package_name'] = 'N/A'
        
    try:
        button_tag = card.find('button', class_='fn-button fn-btn-M fn-btn-primary add-to-card')
        data_items = json.loads(button_tag['data-items'])
        plan['insurance_company'] = data_items['ecommerce']['items'][0]['item_brand']
    except (AttributeError, KeyError, json.JSONDecodeError):
        plan['insurance_company'] = 'N/A'
        
    try:
        plan['price'] = card.find('h1', class_='card-title d-inline-block').text.strip()
    except AttributeError:
        plan['price'] = 'N/A'
        
    try:
        coverage_details = card.find_all('div', class_='row mb4')
        coverage_info = []
        for coverage in coverage_details:
            label = coverage.find('div', class_='col card-text').text.strip()
            value = coverage.find('div', class_='col card-text-end').text.strip()
            coverage_info.append(f"{label}: {value}")
        plan['coverage'] = ", ".join(coverage_info) if coverage_info else 'N/A'
    except AttributeError:
        plan['coverage'] = 'N/A'
    
    try:
        car_model = data_items['ecommerce']['items'][0]['item_name']
        plan['car_model'] = car_model
    except (AttributeError, KeyError, json.JSONDecodeError):
        plan['car_model'] = 'N/A'
    
    insurance_plans.append(plan)

# Convert to DataFrame
extracted_df = pd.DataFrame(insurance_plans)

# Save the DataFrame to a CSV file
extracted_df.to_csv('/Users/apichet/Downloads/insurance_plans2.csv', index=False)

# Create HTML table
html_table = '''
<table border="1">
    <tr>
        <th>Package Name</th>
        <th>Insurance Company</th>
        <th>Price</th>
        <th>Coverage</th>
        <th>Car Model</th>
    </tr>
'''

for plan in insurance_plans:
    html_table += f'''
    <tr>
        <td>{plan['package_name']}</td>
        <td>{plan['insurance_company']}</td>
        <td>{plan['price']}</td>
        <td>{plan['coverage']}</td>
        <td>{plan['car_model']}</td>
    </tr>
    '''

html_table += '</table>'

# Display HTML table
from IPython.display import display, HTML
display(HTML(html_table))

Package Name,Insurance Company,Price,Coverage,Car Model


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Send a GET request to the URL
url = 'https://www.heygoody.com/checkinsurance/plan'
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}")
    exit()

# Step 2: Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Save HTML content to a file for inspection
with open('page_content.html', 'w', encoding='utf-8') as file:
    file.write(soup.prettify())

print("HTML content saved to 'page_content.html' for inspection.")

# Inspect the HTML content to identify correct selectors

HTML content saved to 'page_content.html' for inspection.


In [17]:
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

# อ่านไฟล์ HTML
with open('/Users/apichet/Downloads/เช็คเบี้ยประกันรถยนต์ เปรียบเทียบ รู้ราคาทันที ซื้อง่าย.html', 'r', encoding='utf-8') as file:
    page_content = file.read()

soup = BeautifulSoup(page_content, 'html.parser')

# ตัวอย่างการดึงข้อมูลจาก divs ที่มี class 'card-body'
insurance_plans = []

cards = soup.find_all('div', class_='card-body')
for card in cards:
    package_name = card.find('h1', class_='card-title').get_text(strip=True)
    insurance_company = card.find('span', class_='gtmCarTypeName').get_text(strip=True) if card.find('span', class_='gtmCarTypeName') else 'N/A'
    price = package_name  # ราคาอยู่ใน h1 class='card-title'
    car_model = card.find('span', class_='d-inline-block').get_text(strip=True) if card.find('span', class_='d-inline-block') else 'N/A'

    insurance_plans.append({
        'package_name': package_name,
        'insurance_company': insurance_company,
        'price': price,
        'coverage': 'N/A',  # ยังไม่มีข้อมูล coverage
        'car_model': car_model
    })

# แสดงข้อมูลที่ดึงมา
for plan in insurance_plans:
    print(f"Package Name: {plan['package_name']}")
    print(f"Insurance Company: {plan['insurance_company']}")
    print(f"Price: {plan['price']}")
    print(f"Coverage: {plan['coverage']}")
    print(f"Car Model: {plan['car_model']}")
    print("-" * 40)

# สร้าง DataFrame
df = pd.DataFrame(insurance_plans)

# แสดง DataFrame
display(df)

# สร้าง HTML table
html_table = '''
<table border="1">
<tr>
<th>Package Name</th>
<th>Insurance Company</th>
<th>Price</th>
<th>Coverage</th>
<th>Car Model</th>
</tr>
'''

for plan in insurance_plans:
    html_table += '''
    <tr>
    <td>{}</td>
    <td>{}</td>
    <td>{}</td>
    <td>{}</td>
    <td>{}</td>
    </tr>
    '''.format(plan['package_name'], plan['insurance_company'], plan['price'], plan['coverage'], plan['car_model'])

html_table += '</table>'

# แสดง HTML table
display(HTML(html_table))

""


Package Name,Insurance Company,Price,Coverage,Car Model


In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

# Step 1: Send a GET request to the URL
url = 'https://www.heygoody.com/checkinsurance/car'
response = requests.get(url)

if response.status_code != 200:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}")
    exit()

# Step 2: Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Step 3: Extract the required details
insurance_plans = []

# Adjusting based on observed HTML structure
plan_cards = soup.find_all('div', class_='card fn-card')

# Debugging: Print structure around plan cards
for card in plan_cards:
    print(card.prettify())

# Extracting data from the cards
for card in plan_cards:
    plan = {}
    plan['package_name'] = card.find('h1', class_='card-title').text.strip() if card.find('h1', class_='card-title') else 'N/A'
    plan['insurance_company'] = card.find('span', class_='fn-insurance-Company').text.strip() if card.find('span', class_='fn-insurance-Company') else 'N/A'
    plan['price'] = card.find('h1', class_='card-title').text.strip() if card.find('h1', class_='card-title') else 'N/A'
    plan['coverage'] = card.find('span', class_='card-coverage').text.strip() if card.find('span', class_='card-coverage') else 'N/A'
    plan['car_model'] = card.find('h5', class_='card-subtitle').text.strip() if card.find('h5', class_='card-subtitle') else 'N/A'
    insurance_plans.append(plan)

# Convert to DataFrame
extracted_df = pd.DataFrame(insurance_plans)

# Save the DataFrame to a CSV file
extracted_df.to_csv('/Users/apichet/Downloads/insurance_plans3.csv', index=False)

# Create HTML table
html_table = '''
<table border="1">
<tr>
    <th>Package Name</th>
    <th>Insurance Company</th>
    <th>Price</th>
    <th>Coverage</th>
    <th>Car Model</th>
</tr>
'''

for plan in insurance_plans:
    html_table += '''
    <tr>
        <td>{}</td>
        <td>{}</td>
        <td>{}</td>
        <td>{}</td>
        <td>{}</td>
    </tr>
    '''.format(plan['package_name'], plan['insurance_company'], plan['price'], plan['coverage'], plan['car_model'])

html_table += '</table>'

# Display HTML table
display(HTML(html_table))

Package Name,Insurance Company,Price,Coverage,Car Model


In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

# Step 1: Send a GET request to the URL
url = 'https://www.heygoody.com/checkinsurance/plan'
response = requests.get(url)

if response.status_code != 200:
    print(f"Failed to retrieve the web page. Status code: {response.status_code}")
    exit()

# Step 2: Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Step 3: Extract the required details
insurance_plans = []

# Adjusting based on observed HTML structure
plan_cards = soup.find_all('div', class_='fn-card')

# Debugging: Print structure around plan cards
for card in plan_cards:
    print(card.prettify())

# Extracting data from the cards
for card in plan_cards:
    plan = {}
    plan['package_name'] = card.find('h1', class_='card-title').text.strip() if card.find('h1', class_='card-title') else 'N/A'
    plan['insurance_company'] = card.find('h5', class_='card-subtitle').text.strip() if card.find('h5', class_='card-subtitle') else 'N/A'
    plan['price'] = card.find('div', class_='card-price').text.strip() if card.find('div', class_='card-price') else 'N/A'
    plan['coverage'] = card.find('div', class_='card-coverage').text.strip() if card.find('div', class_='card-coverage') else 'N/A'
    plan['car_model'] = card.find('div', class_='car-model').text.strip() if card.find('div', class_='car-model') else 'N/A'
    insurance_plans.append(plan)

# Convert to DataFrame
extracted_df = pd.DataFrame(insurance_plans)

# Save the DataFrame to a CSV file
extracted_df.to_csv('/Users/apichet/Downloads/insurance_plans3.csv', index=False)

# Create HTML table
html_table = '''
<table border="1">
<tr>
    <th>Package Name</th>
    <th>Insurance Company</th>
    <th>Price</th>
    <th>Coverage</th>
    <th>Car Model</th>
</tr>
'''

for plan in insurance_plans:
    html_table += '''
    <tr>
        <td>{}</td>
        <td>{}</td>
        <td>{}</td>
        <td>{}</td>
        <td>{}</td>
    </tr>
    '''.format(plan['package_name'], plan['insurance_company'], plan['price'], plan['coverage'], plan['car_model'])

html_table += '</table>'

# Display HTML table
display(HTML(html_table))

Package Name,Insurance Company,Price,Coverage,Car Model
